In [4]:
from collections import Counter
import random
import string
import difflib
import time

**Read the data and prepare it for further use.**


In [5]:
data = []
vocab = []
total_count = 0
pair_prob = {}
with open('w2_.txt', 'r', encoding='latin-1') as f:
  data = f.readlines()
  for i in data:
    l =  i.rstrip()
    temp = l.split('\t')
    vocab.append(temp[1])
    vocab.append(temp[2])
    pair_prob[f"{temp[1]} {temp[2]}"] = temp[0]
    total_count += int(temp[0])

In [6]:
WORDS = Counter(vocab)

**Find the probability of each pair of word**

In [7]:
for key in pair_prob:
  pair_prob[key] = int(pair_prob[key])/total_count

**Functions for creating all possible versions of word with 1 or 2 changes.**

In [8]:
def edits1(word):
    "All edits that are one edit away from `word`."
    letters    = 'abcdefghijklmnopqrstuvwxyz'
    splits     = [(word[:i], word[i:])    for i in range(len(word) + 1)]
    deletes    = [L + R[1:]               for L, R in splits if R]
    transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R)>1]
    replaces   = [L + c + R[1:]           for L, R in splits if R for c in letters]
    inserts    = [L + c + R               for L, R in splits for c in letters]
    return set(deletes + transposes + replaces + inserts)

def edits2(word):
    "All edits that are two edits away from `word`."
    return (e2 for e1 in edits1(word) for e2 in edits1(e1))

In [9]:
word = 'lkkes'
t = edits1(word)
for i in t:
  if i in WORDS:
    print(i, WORDS[i])

likes 67
lakes 48


In [10]:
def first_word(word):
  t = edits1(word)
  s = set(edits2(word))
  t = t.union(s)
  if word in WORDS:
    return word
  word = ''
  max = -10
  for i in t:
    if i in WORDS:
      if int(WORDS[i]) > max:
        max = int(WORDS[i])
        word = i
  return word
first_word('we')

'we'

Function for correction for word based on previous word.

In [11]:
def common_case(word_prev, word):
 # Generate possible edits for the given word using edits1 and edits2 functions
 t = edits1(word)
 s = edits2(word)
 # Initialize lists to store possible words based on edits
 possible_words = []
 possible_words_2 = []

 # Initialize final word as an empty string
 final_word = ''
 # Construct a pair of words for checking in pair_prob
 pair_1 = word_prev + ' ' + word
 # If the pair is already in pair_prob, return the word immediately
 if pair_1 in pair_prob:
    return word

 # Iterate through edits1 results and check if they are in WORDS
 for i in t:
    if i in WORDS:
      possible_words.append(i)
 # Iterate through edits2 results and check if they are in WORDS
 for i in s:
    if i in WORDS:
      possible_words_2.append(i)

 # Initialize variables to track the maximum probability and word frequency
 max = -10
 max_word = -10
 max_word2 = -10
 # Iterate through possible words from edits1
 for word_ in possible_words:
    # Construct a pair with the previous word and the current possible word
    pair = word_prev +' ' + word_
    # If the pair is in pair_prob, update max and final_word if necessary
    if pair in pair_prob:
      if int(pair_prob[pair]) > max:
        max = int(pair_prob[pair])
        final_word = pair.split()[1]
      else:
        if max_word < WORDS[pair.split()[1]]:
          max_word = WORDS[pair.split()[1]]
          final_word = pair.split()[1]

 # If no final word is found yet, iterate through possible words from edits2
 if final_word == '':
    for word_2 in possible_words_2:
      pair = word_prev +' ' + word_2
      # If the pair is in pair_prob, update max and final_word if necessary
      if pair in pair_prob:
        if int(pair_prob[pair]) > max:
          max = int(pair_prob[pair])
          final_word = pair.split()[1]
        else:
          if max_word < WORDS[pair.split()[1]]:
            max_word = WORDS[pair.split()[1]]
            final_word = pair.split()[1]
      else:
        # If the pair is not in pair_prob, check word frequency
        if max_word2 < WORDS[word_2]:
          max_word2 = WORDS[word_2]
          final_word = word_2

 # Return the final word determined by the logic above
 return final_word


**Generating a test set with one mistake per word to evaluate the algorithm.**


In [12]:
sentences1 = [
    "The big tree sways in the wind",
    "She runs fast to catch the bus",
    "The cat sleeps on the mat",
    "He eats an apple for lunch",
    "The sun shines brightly in the sky",
    "They play games at the park",
    "I drink water when I'm thirsty",
    "We walk together in the park",
    "She reads books every night",
    "He watches TV after dinner"
]

sentences = []
for sen in sentences1:
    sentences.append(sen.split())
test_sentences = []

for i in range(10):
    for j in range(10):
        new = sentences[i][0].lower()
        for k in range(1, len(sentences[i])):
            new += " " + random.choice(list(edits1(sentences[i][k]))).lower()
        test_sentences.append(new)

In [13]:
# test_sentences

Function to evaluate similarity of two sentences

In [14]:
def similarity(s1, s2):
    normalized1 = s1.lower()
    normalized2 = s2.lower()
    matcher = difflib.SequenceMatcher(None, normalized1, normalized2)
    return matcher.ratio()


In [15]:
print("Test for Dataset with 1 error in word")
count = len(test_sentences)
summator = 0
for k in range(len(test_sentences)):
  w = ''
  prev_word = ''
  for i in range(len(test_sentences[k].split())):
    start = time.time()
    if i == 0:
      w += first_word(test_sentences[k].split()[i]) + ' '
      prev_word = first_word(test_sentences[k].split()[i])
    else:
      w += common_case(prev_word, test_sentences[k].split()[i])+ ' '
      prev_word = common_case(prev_word, test_sentences[k].split()[i])

  print("Sentence before: ", test_sentences[k])
  print("Sentence after: ", w)
  print("Accuracy", similarity( sentences1[k // 10], w))
  print("Time for this sentence", time.time() - start)
  summator += similarity( sentences1[k // 10], w)

print("overall precision", summator/count)

Test for Dataset with 1 error in word
Sentence before:  the btig itree swanys din thei winod
Sentence after:  the big tree stands in the wind 
Accuracy 0.9032258064516129
Time for this sentence 0.15904808044433594
Sentence before:  the bvg tee swiays rin tse winnd
Sentence after:  the big the says in the wind 
Accuracy 0.9152542372881356
Time for this sentence 0.15616607666015625
Sentence before:  the brig twree swsays ia thi wiod
Sentence after:  the brig were ways in the wild 
Accuracy 0.8524590163934426
Time for this sentence 0.11167478561401367
Sentence before:  the bwig trze swans inr twe wqind
Sentence after:  the big tree says in the wind 
Accuracy 0.9666666666666667
Time for this sentence 0.15488791465759277
Sentence before:  the jig trew msways ibn ithe winz
Sentence after:  the jig the always in the win 
Accuracy 0.8333333333333334
Time for this sentence 0.06030726432800293
Sentence before:  the tbig trqe swafys it tbhe hind
Sentence after:  the big tree says it the hind 
Acc

**Generating a test set with two mistake per word to evaluate the algorithm.**


In [16]:
sentences1 = [
    "The big tree sways in the wind",
    "She runs fast to catch the bus",
    "The cat sleeps on the mat",
    "He eats an apple for lunch",
    "The sun shines brightly in the sky",
    "They play games at the park",
    "I drink water when I'm thirsty",
    "We walk together in the park",
    "She reads books every night",
    "He watches TV after dinner"
]

sentences = []
for sen in sentences1:
    sentences.append(sen.split())
test_sentences = []

for i in range(10):
    for j in range(10):
        new = sentences[i][0].lower()
        for k in range(1, len(sentences[i])):
            new += " " + random.choice(list(edits1(sentences[i][k]))+ list(edits2(sentences[i][k]))).lower()
        test_sentences.append(new)

**Now we will try to fix words with 2 errors.**

In [ ]:
print("Test for Dataset with 2 error in word")
count = len(test_sentences)
summator = 0
for k in range(len(test_sentences)):
  w = ''
  prev_word = ''
  for i in range(len(test_sentences[k].split())):
    start = time.time()
    if i == 0:
      w += first_word(test_sentences[k].split()[i]) + ' '
      prev_word = first_word(test_sentences[k].split()[i])
    else:
      w += common_case(prev_word, test_sentences[k].split()[i])+ ' '
      prev_word = common_case(prev_word, test_sentences[k].split()[i])

  print("Sentence before: ", test_sentences[k])
  print("Sentence after: ", w)
  print("Accuracy", similarity( sentences1[k // 10], w))
  print("Time for this sentence", time.time() - start)
  summator += similarity( sentences1[k // 10], w)

print("overall precision", summator/count)


**Justifying my decision:**

I use Bigrams and my result is following:
~85% accuracy for words with 2 errors (i think it is very good results because some words i cannot recognize by myself)
~90% accuracy for words with 1 error

I refrained from utilizing trigrams and higher-order n-grams due to several considerations:

1. **Sufficiency of Bigrams**: Bigrams are sufficient for understanding context and achieving satisfactory results. This approach is not only effective but also computationally efficient, making it a preferable choice for the task at hand.

2. **Performance and Efficiency**: Bigrams offer a balance between performance and computational resources. While trigrams and higher-order n-grams can provide more detailed context, they come at the cost of increased computational complexity and potentially diminishing returns in terms of accuracy improvements.

3. **Contextual Relevance**: For many applications, the context provided by bigrams is sufficient to make accurate predictions or corrections. Higher-order n-grams may capture more nuanced contexts, but this additional detail is often unnecessary for the task of spelling correction.

4. **Simplicity and Maintainability**: Bigrams simplify the model, making it easier to understand, implement, and maintain. This simplicity is particularly valuable in applications where the model's complexity needs to be kept to a minimum.

In summary, the decision to use bigrams instead of trigrams or higher-order n-grams is based on a combination of factors, including the sufficiency of bigrams for the task, the trade-off between performance and computational efficiency, the relevance of the context captured by bigrams, and the benefits of simplicity and maintainability.



Norwig's solutions

In [19]:
import re
from collections import Counter

def words(text): return re.findall(r'\w+', text.lower())

def P(word, N=sum(WORDS.values())):
    "Probability of `word`."
    return WORDS[word] / N

def correction(word):
    "Most probable spelling correction for word."
    return max(candidates(word), key=P)

def candidates(word):
    "Generate possible spelling corrections for word."
    return (known([word]) or known(edits1(word)) or known(edits2(word)) or [word])

def known(words):
    "The subset of `words` that appear in the dictionary of WORDS."
    return set(w for w in words if w in WORDS)

def edits1(word):
    "All edits that are one edit away from `word`."
    letters    = 'abcdefghijklmnopqrstuvwxyz'
    splits     = [(word[:i], word[i:])    for i in range(len(word) + 1)]
    deletes    = [L + R[1:]               for L, R in splits if R]
    transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R)>1]
    replaces   = [L + c + R[1:]           for L, R in splits if R for c in letters]
    inserts    = [L + c + R               for L, R in splits for c in letters]
    return set(deletes + transposes + replaces + inserts)

def edits2(word):
    "All edits that are two edits away from `word`."
    return (e2 for e1 in edits1(word) for e2 in edits1(e1))

In [20]:
print("Test for Norwig's solution on Dataset with 2 error in word")
count = len(test_sentences)
summator = 0
for k in range(len(test_sentences)):
  w = ''
  prev_word = ''
  for i in range(len(test_sentences[k].split())):
    start = time.time()
    w += correction(test_sentences[k].split()[i])

  print("Sentence before: ", test_sentences[k])
  print("Sentence after: ", w)
  print("Accuracy", similarity( sentences1[k // 10], w))
  print("Time for this sentence", time.time() - start)
  summator += similarity( sentences1[k // 10], w)

print("overall precision", summator/count)


Test for Norwig's solution on Dataset with 2 error in word
Sentence before:  the bbq treewa suwans iwi btqhe wilnd
Sentence after:  thebbqtreesswansiithewind
Accuracy 0.7272727272727273
Time for this sentence 0.002976655960083008
Sentence before:  the biwm trce cswags iqe whq wind
Sentence after:  thehimtrueswagsicewhowind
Accuracy 0.6181818181818182
Time for this sentence 0.00042176246643066406
Sentence before:  the bike tryeh swtaeys ipl thwm winok
Sentence after:  thebiketruestaysillthemwink
Accuracy 0.6666666666666666
Time for this sentence 0.0009052753448486328
Sentence before:  the bfpg xtreme wswhys san khn windwp
Sentence after:  thebigextremeswayssankenwindup
Accuracy 0.7
Time for this sentence 0.0038352012634277344
Sentence before:  the clig wrtee snayhs inm shf wamnd
Sentence after:  theclipweresaysinshewand
Accuracy 0.6296296296296297
Time for this sentence 0.003052949905395508
Sentence before:  the btgi trdee mwayos inbs tnhv jrind
Sentence after:  thebigtreewaysinsthegrin

Overall precision ~77% for Norwig's solution against ~85% for Bigramms
